In [13]:
#code reference: https://www.pyimagesearch.com/2014/09/01/build-kick-ass-mobile-document-scanner-just-5-minutes/
#https://www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/

from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils

def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect


image = cv2.imread("metro_card.jpeg")
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)
cv2.imshow("Original1", imutils.resize(image, height = 500))
cv2.waitKey(0)

# convert the image to grayscale, blur it, and find edges
# in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)

# show the original image and the edge detected image
#cv2.imshow("Image", image)
cv2.putText(edged, "Edged img", (0, 0),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
cv2.imshow("OCR1", edged)
cv2.waitKey(0)
#cv2.destroyAllWindows()


cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

# loop over the contours
for c in cnts:
# approximate the contour
    peri = cv2.arcLength(c, True) 
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)


    if len(approx) == 4:
        screenCnt = approx
        break

# show the contour (outline) of the piece of paper
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("OCR2", image)
cv2.waitKey(0)
#cv2.destroyAllWindows()

arr=np.zeros((4, 2), dtype = "float32")
cnt=0
for i in approx:
    for j in i:
        arr[cnt] = [j[0], j[1]]
        cnt+=1
        

pts1 = arr
rect = order_points(pts1)
(tl, tr, br, bl) = rect
widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
maxWidth = max(int(widthA), int(widthB))
heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
maxHeight = max(int(heightA), int(heightB))
dst = np.array([
[0, 0],
[maxWidth - 1, 0],
[maxWidth - 1, maxHeight - 1],
[0, maxHeight - 1]], dtype = "float32")
dummyDst = dst.astype('int32')
dummyImg = image.copy()
cv2.drawContours(dummyImg, [dummyDst], -1, (0, 0, 255), 2)
cv2.imshow("OCR3", dummyImg)
cv2.waitKey(0)
#cv2.destroyAllWindows()
M = cv2.getPerspectiveTransform(rect, dst)
warped = cv2.warpPerspective(image.copy(), M, (maxWidth, maxHeight))
cv2.imshow("OCR4", warped)
cv2.waitKey(0)
#cv2.destroyAllWindows()

warped_bw = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped_bw, 9, offset = 11, method= 'gaussian')
#warped_bw = (warped_bw > T).astype("uint8") * 255
cv2.imshow("OCR5", imutils.resize(warped_bw, height = 250))
cv2.waitKey(0)
#cv2.destroyAllWindows()
cv2.imshow("Original2", imutils.resize(image, height = 500))
cv2.waitKey(0)
cv2.destroyAllWindows()